# 한글-Claude-v2 Model: Conversational Interface - Chatbot with Claude LLM

이 노트북은 SageMaker Studio의**`데이터 과학 3.0`** 커널과 잘 작동해야 합니다.

이 노트북에서는 Amazon Bedrock의 기본 모델 (FM) 을 사용하여 챗봇을 구축할 것입니다.사용 사례에서는 Claude를 챗봇 구축을 위한 FM으로 사용합니다.

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 개요

챗봇 및 가상 어시스턴트와 같은 대화형 인터페이스를 사용하여 고객의 사용자 경험을 향상시킬 수 있습니다. 챗봇은 자연어 처리 (NLP) 및 기계 학습 알고리즘을 사용하여 사용자 쿼리를 이해하고 이에 응답합니다.챗봇은 고객 서비스, 판매, 전자 상거래와 같은 다양한 애플리케이션에서 사용되어 사용자에게 빠르고 효율적인 응답을 제공할 수 있습니다.웹사이트, 소셜 미디어 플랫폼 및 메시징 앱과 같은 다양한 채널을 통해 액세스할 수 있습니다.


## Amazon Bedrock을 사용하는 챗봇

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## 사용 사례
- 1.**챗봇 (기본)** - FM 모델을 사용하는 제로샷 챗봇
- 2.**프롬프트를 사용하는 챗봇** - 템플릿 (Langchain) - 프롬프트 템플릿에 일부 컨텍스트가 제공된 챗봇
- 3.**페르소나가 있는 챗봇** - 정의된 역할을 가진 챗봇. 즉, 커리어 코치와 인간 상호작용
- 4.**컨텍스트 인식 챗봇** - 임베딩을 생성하여 외부 파일을 통해 컨텍스트를 전달합니다.

## Amazon Bedrock으로 챗봇을 구축하기 위한 랭체인 프레임워크
챗봇과 같은 대화형 인터페이스에서는 단기적 수준뿐만 아니라 장기적 수준에서도 이전 상호 작용을 기억하는 것이 매우 중요합니다.

LangChain은 메모리 구성 요소를 두 가지 형태로 제공합니다.먼저 LangChain은 이전 채팅 메시지를 관리하고 조작하기 위한 도우미 유틸리티를 제공합니다.모듈식으로 설계되어 사용 방식에 관계없이 유용하게 사용할 수 있습니다.둘째, LangChain은 이러한 유틸리티를 체인에 통합하는 쉬운 방법을 제공합니다.
이를 통해 다양한 유형의 추상화를 쉽게 정의하고 상호 작용할 수 있으므로 강력한 챗봇을 쉽게 구축할 수 있습니다.

## 컨텍스트를 활용한 챗봇 구축하기 - 핵심 요소

컨텍스트 인식 챗봇을 구축하는 첫 번째 프로세스는 컨텍스트에 대한**임베딩을 생성**하는 것입니다.일반적으로 임베딩 모델을 통해 실행되고 일종의 벡터 저장소에 저장될 임베딩을 생성하는 통합 프로세스가 있습니다.이 예제에서는 이를 위해 GPT-J 임베딩 모델을 사용하고 있습니다.

![Embeddings](./images/embeddings_lang.png)

두 번째 프로세스는 사용자 요청 오케스트레이션, 상호 작용, 호출 및 결과 반환입니다.

![Chatbot](./images/chatbot_lang.png)

## 아키텍처 [컨텍스트 인식 챗봇]
![4](./images/context-aware-chatbot.png)


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.


이 노트북에는 몇 가지 추가 종속성도 필요합니다.

- [FAISS](https://github.com/facebookresearch/faiss), 벡터 임베딩 저장용
- [iPyWidgets](https://ipywidgets.readthedocs.io/en/stable/), 노트북의 대화형 UI 위젯용
- [PyPDF](https://pypi.org/project/pypdf/), PDF 파일 처리용

In [2]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" langchain==0.0.249 "pypdf>=3.8,<4"

Note: you may need to restart the kernel to use updated packages.


## 1. Bedrock Client 생성

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## 2. 챗봇(기본 - 컨텍스트 없음)

우리는 LangChain의 [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain)을 사용하여 시작합니다. 대화. 또한 메시지 저장을 위해 [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html)를 사용합니다. 메시지 목록으로 기록을 얻을 수도 있습니다(채팅 모델에서 매우 유용합니다).

챗봇은 이전 상호작용을 기억해야 합니다. 대화 기억을 통해 우리는 그렇게 할 수 있습니다. 대화형 메모리를 구현하는 방법에는 여러 가지가 있습니다. LangChain의 맥락에서 이들은 모두 ConversationChain 위에 구축됩니다.

**참고:** 모델 출력은 비결정적입니다.

In [4]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory

cl_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

print_ww(conversation.predict(input="안녕하세요?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 안녕하세요?
AI:

> Finished chain.
 네, 안녕하세요! 만나서 반갑습니다.


### 결과 분석

여기서 무슨 일이 일어나는가? 우리는 "안녕하세요!"라고 말했습니다. 모델은 몇 가지 대화를 나눴습니다. 이는 Langchain ConversationChain에서 사용하는 기본 프롬프트가 Claude에 맞게 잘 설계되지 않았기 때문입니다. 
- [효과적인 클로드 프롬프트](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)는 `\n\nHuman\n\nAassistant:`로 끝나야 합니다. 이 문제를 해결해 보겠습니다.

Claude의 프롬프트 작성 방법에 대해 자세히 알아보려면 [Anthropic 문서](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)를 확인하세요.

## 3. 프롬프트 템플릿(Langchain)을 이용한 챗봇

LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 기능을 제공합니다. [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) 클래스를 사용하여 f-string 템플릿에서 프롬프트를 구성하겠습니다.

In [5]:
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude

claude_prompt = PromptTemplate.from_template("""다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
{history}


Human: {input}


Assistant:
""")

# claude_prompt = PromptTemplate.from_template("""다음은 사람과 AI의 친근한 대화입니다.AI는 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. AI가 모른다면
# 질문에 대한 대답은 솔직히 모른다고 말합니다.

# Current conversation:
# {history}


# Human: {input}


# Assistant:
# """)

print("claude_prompt: \n", claude_prompt)

claude_prompt: 
 input_variables=['history', 'input'] output_parser=None partial_variables={} template='다음은 사람과 인공지능의 친근한 대화입니다. \n인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. \n인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.\n\nCurrent conversation:\n{history}\n\n\nHuman: {input}\n\n\nAssistant:\n' template_format='f-string' validate_template=True


In [6]:
conversation.prompt = claude_prompt

print_ww(conversation.predict(input="안녕하세요?"))

 안녕하세요! 반갑습니다.


#### (1) 새로운 질문

모델이 초기 메시지로 응답했습니다. 몇 가지 질문을 해보겠습니다.

In [7]:
print_ww(conversation.predict(input="새로운 정원을 시작하는 방법에 대한 몇 가지 팁을 알려주세요."))

 네, 새로운 정원을 시작하는 데 도움이 되는 몇 가지 팁을 알려드리겠습니다.

- 먼저 정원의 위치를 잘 선택하세요. 해당 장소가 충분한 햇빛을 받는지, 물 공급이 용이한지 확인하세요.

- 토양의 상태를 테스트하여 pH 수준과 영양분을 확인하세요. 필요하다면 토양 개선제를 추가하세요.

- 정원 계획을 세우세요. 어떤 종류의 식물을 키울지, 어디에 심을지 미리 계획하세요.

- 해당 식물에 맞는 토질, 일조량, 물, 영양분을 공급하세요.

- 제초와 병충해 관리를 상시 확인하세요.

- 수확기에는 과수나 채소를 정기적으로 관리, 수확하세요.

- 퇴비나 유기물을 추가하여 토양을 계속 개선하세요.

- 정원 일지를 작성하여 식물의 성장과정을 기록하세요.

이런 팁들을 염두에 두고 천천히 정원 가꾸기를 즐기세요. 많은 시행착오 끝에 아름다운 정원이 꽃피리라 믿습니다.


#### (2) 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문해 보겠습니다.

In [8]:
print_ww(conversation.predict(input="좋아요. 토마토에도 어울릴까요?"))

 네, 그 팁들은 토마토 정원 가꾸기에도 도움이 될 것입니다.

토마토를 재배하기 위한 몇 가지 추가 팁을 제안드리겠습니다:

- 토마토는 일조량이 많이 필요하므로 정남향의 순수한 햇빛을 받는 장소를 선택하세요.

- 토마토는 따뜻한 온도를 선호하므로 한여름 내내 섭씨 21-27도를 유지할 수 있는 정원을 만드세요.

- 토마토는 가능한 한 햇볕이 직접 닿는 곳에 심으세요.

- 토마토는 수분과 영양분 섭취가 많으므로 토양에 퇴비를 골고루 섞어주세요.

- 정기적으로 관수하되 뿌리 썩음이 생기지 않도록 주의하세요.

- 지지대를 설치하여 식물이 땅에 닿지 않도록 하세요.

- 과일이 익으면 수확 후 새 싹이 자라도록 지속적으로 관리하세요.

토마토 재배에 성공적인 정원 만들기를 기원합니다!


#### (3) 대화를 마치며

In [9]:
print_ww(conversation.predict(input="그게 다야, 고마워!"))

 네, 처음 새 정원 만들기에 관한 제안과 토마토 재배 팁을 드렸습니다. 정보가 도움이 되었길 바랍니다. 새 정원 가꾸기에 성공하시기를 응원합니다!


#### (4) 실습: 
- 클로드는 아직도 말이 많아요. Claude가 더 짧은 답변을 제공하도록 프롬프트를 변경해 보세요.

## 4. ipywidgets를 사용한 대화형 세션

다음 유틸리티 클래스를 사용하면 Claude와 보다 자연스러운 방식으로 상호 작용할 수 있습니다. 입력창에 질문을 적고 클로드의 답변을 받습니다. 그러면 대화를 계속할 수 있습니다.

In [10]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

#### (1) 채팅을 시작해 보겠습니다. 다음 질문을 테스트할 수도 있습니다.

1. 농담 하나 해줘
2. 또 다른 농담을 들려주세요
3. 첫 번째 농담은 무엇이었나요?
4. 첫 번째 농담과 같은 주제로 또 다른 농담을 할 수 있나요?

위의 4가지를 순서대로 아래에 입력하시고, "Send" 버튼을 눌러 보세요.

In [11]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

## 5.페르소나를 활용한 챗봇

AI 비서가 커리어 코치 역할을 하게 됩니다. 
- 역할극 대화에서는 채팅을 시작하기 전에 사용자 메시지를 설정해야 합니다. ConversationBufferMemory는 대화 상자를 미리 채우는 데 사용됩니다.

### (1) ConversationChain 생성 필요한 메모리 초기화, Bedrock Claude 설정

In [12]:
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다")
memory.chat_memory.add_ai_message("나는 직업 코치이며 직업에 대한 조언을 제공합니다")
cl_llm = Bedrock(model_id="anthropic.claude-v2",
                 client=boto3_bedrock,
                 model_kwargs={"max_tokens_to_sample": 1000},                 
                )
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
{history}


Human: {input}


Assistant:
""")


print("claude_prompt: \n", claude_prompt)
conversation.prompt = claude_prompt

claude_prompt: 
 input_variables=['history', 'input'] output_parser=None partial_variables={} template='다음은 사람과 인공지능의 친근한 대화입니다. \n인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. \n인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.\n\nCurrent conversation:\n{history}\n\n\nHuman: {input}\n\n\nAssistant:\n' template_format='f-string' validate_template=True


### (2) 인공지능 관련 직업 질문

In [13]:
# print_ww(conversation.predict(input="“AI의 직업 종류는 무엇입니까?"))
print_ww(conversation.predict(input="“인공지능에 관련된 직업은 어떤 것이 있습니까?"))



> Entering new ConversationChain chain...
Prompt after formatting:
다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
Human: 당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다
AI: 나는 직업 코치이며 직업에 대한 조언을 제공합니다


Human: “인공지능에 관련된 직업은 어떤 것이 있습니까?


Assistant:


> Finished chain.
 인공지능에 관련된 직업은 다음과 같습니다.

- 인공지능 연구원 - 인공지능 알고리즘과 기술을 연구하고 개발합니다. 수학, 통계, 컴퓨터 공학 등의 배경 지식이 필요합니다.

- 인공지능 엔지니어 - 인공지능 시스템을 설계, 구축, 테스트, 유지보수 합니다. 소프트웨어 엔지니어링 기술이 필요합니다.

- 데이터 사이언티스트 - 대용량의 데이터를 수집, 가공, 분석하고 인사이트를 도출합니다. 통계, 머신러닝, 데이터 마이닝 등의 기술이 필요합니다.

- 머신러닝 엔지니어 - 머신러닝 모델을 개발하고 실제 서비스에 적용합니다. 머신러닝 알고리즘에 대한 지식이 필요합니다.

- NLP(자연어처리) 엔지니어 - 자연어를 분석하고 이해하는 인공지능 시스템을 개발합니다. 자연어 처리와 자연어 이해 기술이 필요합니다.

- 로보틱스 엔지니어 - 인공지능 기반의 로봇을 설계, 개발, 제어합니다. 로봇공학, 제어이론, 인공지능 등의 기술이 필요합니다.

이외에도 비즈니스 분석가, 프로덕트 매니저 등 인공지능 기술을 활용하는 다양한 직업군이 있습니다. 전문성과 관심분야에 따라 적합한 인공지능 관련 직업을 탐색하는 것이 좋습니다.


### (3) 인공지능 관련 직업데 대한 세부 질문

In [14]:
print_ww(conversation.predict(input="이 직업들은 실제로 무엇을 하는가요? 재미있나요?"))



> Entering new ConversationChain chain...
Prompt after formatting:
다음은 사람과 인공지능의 친근한 대화입니다. 
인공지능은 말이 많고 상황에 맞는 구체적인 세부 정보를 많이 제공합니다. 
인공지능은 모른다면 질문에 대한 대답은 솔직히 모른다고 말합니다.

Current conversation:
Human: 당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다
AI: 나는 직업 코치이며 직업에 대한 조언을 제공합니다
Human: “인공지능에 관련된 직업은 어떤 것이 있습니까?
AI:  인공지능에 관련된 직업은 다음과 같습니다.

- 인공지능 연구원 - 인공지능 알고리즘과 기술을 연구하고 개발합니다. 수학, 통계, 컴퓨터 공학 등의 배경 지식이 필요합니다. 

- 인공지능 엔지니어 - 인공지능 시스템을 설계, 구축, 테스트, 유지보수 합니다. 소프트웨어 엔지니어링 기술이 필요합니다.

- 데이터 사이언티스트 - 대용량의 데이터를 수집, 가공, 분석하고 인사이트를 도출합니다. 통계, 머신러닝, 데이터 마이닝 등의 기술이 필요합니다. 

- 머신러닝 엔지니어 - 머신러닝 모델을 개발하고 실제 서비스에 적용합니다. 머신러닝 알고리즘에 대한 지식이 필요합니다.

- NLP(자연어처리) 엔지니어 - 자연어를 분석하고 이해하는 인공지능 시스템을 개발합니다. 자연어 처리와 자연어 이해 기술이 필요합니다.

- 로보틱스 엔지니어 - 인공지능 기반의 로봇을 설계, 개발, 제어합니다. 로봇공학, 제어이론, 인공지능 등의 기술이 필요합니다.

이외에도 비즈니스 분석가, 프로덕트 매니저 등 인공지능 기술을 활용하는 다양한 직업군이 있습니다. 전문성과 관심분야에 따라 적합한 인공지능 관련 직업을 탐색하는 것이 좋습니다.


Human: 이 직업들은 실제로 무엇을 하는가요? 재미있나요?


Assistant:


> Finished chain.
 네, 인공지능 관련 직업들은 구체적으로 어

## 6.Next Action

프롬프트를 "조정"하여 다소 자세한 답변을 얻을 수 있습니다. 예를 들어 문장 수를 변경하거나 해당 명령을 모두 제거해 보세요. 전체 답변을 얻으려면 'max_tokens_to_sample' 수(예: 1000 또는 2000)를 변경해야 할 수도 있습니다.

### 요약
- 이 데모에서는 Claude LLM을 사용하여 다음 패턴으로 대화형 인터페이스를 만들었습니다.

1. 챗봇(기본 - 맥락 없음)

2. 프롬프트 템플릿(Langchain)을 이용한 챗봇

3. 페르소나를 갖춘 챗봇